In [1]:
using DelimitedFiles
using LinearAlgebra

In [2]:
A = readdlm("D:/julia-workspace/admission.txt", ',', Float64, '\n')

100×3 Array{Float64,2}:
 34.6237  78.0247  0.0
 30.2867  43.895   0.0
 35.8474  72.9022  0.0
 60.1826  86.3086  1.0
 79.0327  75.3444  1.0
 45.0833  56.3164  0.0
 61.1067  96.5114  1.0
 75.0247  46.554   1.0
 76.0988  87.4206  1.0
 84.4328  43.5334  1.0
 95.8616  38.2253  0.0
 75.0137  30.6033  0.0
 82.3071  76.482   1.0
  ⋮                
 78.6354  96.6474  1.0
 52.348   60.7695  0.0
 94.0943  77.1591  1.0
 90.4486  87.5088  1.0
 55.4822  35.5707  0.0
 74.4927  84.8451  1.0
 89.8458  45.3583  1.0
 83.4892  48.3803  1.0
 42.2617  87.1039  1.0
 99.315   68.7754  1.0
 55.34    64.9319  1.0
 74.7759  89.5298  1.0

In [3]:
y = A[:,end]

100-element Array{Float64,1}:
 0.0
 0.0
 0.0
 1.0
 1.0
 0.0
 1.0
 1.0
 1.0
 1.0
 0.0
 0.0
 1.0
 ⋮
 1.0
 0.0
 1.0
 1.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [4]:
X = [ones(length(y)) A[:,1:end-1]]

100×3 Array{Float64,2}:
 1.0  34.6237  78.0247
 1.0  30.2867  43.895
 1.0  35.8474  72.9022
 1.0  60.1826  86.3086
 1.0  79.0327  75.3444
 1.0  45.0833  56.3164
 1.0  61.1067  96.5114
 1.0  75.0247  46.554
 1.0  76.0988  87.4206
 1.0  84.4328  43.5334
 1.0  95.8616  38.2253
 1.0  75.0137  30.6033
 1.0  82.3071  76.482
 ⋮             
 1.0  78.6354  96.6474
 1.0  52.348   60.7695
 1.0  94.0943  77.1591
 1.0  90.4486  87.5088
 1.0  55.4822  35.5707
 1.0  74.4927  84.8451
 1.0  89.8458  45.3583
 1.0  83.4892  48.3803
 1.0  42.2617  87.1039
 1.0  99.315   68.7754
 1.0  55.34    64.9319
 1.0  74.7759  89.5298

In [5]:
(N,D) = size(X)

(100, 3)

In [6]:
function sigmoid(a)
   β = 1/(1+ℯ^(-a)) 
end

sigmoid (generic function with 1 method)

In [51]:
function hessian(X,θ)
    N, D = size(X)
    a = sigmoid.(X*θ)
    b = a .* (1 .- a)
    c = map(t->(X[t,:]*X[t,:]'),1:N)
    H = sum(b.*c)
#     c = Diagonal(b)
#     H = X'*c*X
    H
end

hessian (generic function with 1 method)

In [52]:
θ = zeros(D)
hessian(X,θ)

3×3 Array{Float64,2}:
   25.0   1641.11       1655.55
 1641.11     1.171e5       1.08466e5
 1655.55     1.08466e5     1.1818e5

In [53]:
function newTon_Raphson(X,maxIterations=10)
    N, D = size(X)
    θ = zeros(D)
    θs = []
    for t = 1:maxIterations
        b = sigmoid.(X*θ)
        ∇= X'*(b-y)/N
        H = hessian(X,θ)/N
        θ = θ - H^-1*∇
        push!(θs,θ)
    end
    θs
end

newTon_Raphson (generic function with 2 methods)

In [54]:
θs=newTon_Raphson(X,20)

20-element Array{Any,1}:
 [-7.189987744313788, 0.05936349904614033, 0.055768687906594755]
 [-12.439101383661573, 0.10290118777438229, 0.09734410214438814]
 [-18.026680156056358, 0.14857532168427448, 0.14284277461201994]
 [-22.7458177396772, 0.18678317031595496, 0.1816170265712656]
 [-24.868663274057734, 0.20388015094431275, 0.1990698448658108]
 [-25.15692672592341, 0.20619632901139512, 0.2014355245565627]
 [-25.16133256589908, 0.20623170525772586, 0.20147159227081424]
 [-25.161333566639534, 0.20623171329398302, 0.20147160044196347]
 [-25.16133356663959, 0.20623171329398346, 0.20147160044196394]
 [-25.16133356663956, 0.20623171329398318, 0.20147160044196366]
 [-25.161333566639538, 0.20623171329398304, 0.20147160044196352]
 [-25.161333566639556, 0.20623171329398318, 0.20147160044196366]
 [-25.161333566639573, 0.2062317132939833, 0.2014716004419638]
 [-25.161333566639566, 0.20623171329398327, 0.20147160044196374]
 [-25.161333566639552, 0.20623171329398315, 0.20147160044196363]
 [-25.16133

In [11]:
function classify(X, θ)
    h = X'*θ
    if (h<0) 0.0 else 1.0 end
end

classify (generic function with 1 method)

In [12]:
prediction = map(i->classify(X[i,:], θs[end,end]), 1:N)
accuracy = sum(y .== prediction)/N

0.89

In [13]:
using Optim

In [14]:
function cost(θ::Array{Float64})::Float64
    N = length(y)
    h = sigmoid.(X*θ)
    l = y'*log.(h)+(1 .- y)'*log.(1 .- h)
    J = -l/N + 0.
    J
end

cost (generic function with 1 method)

In [15]:
function g!(∇, θ)
    N = length(y)
    h = sigmoid.(X*θ)
    temp = X'*(h - y)/N .+ 0.
    for j = 1:length(temp)
        ∇[j] = temp[j]
    end
end

g! (generic function with 1 method)

In [55]:
function h!(H,θ)
    N, D = size(X)
    a = sigmoid.(X*θ)
    b = a.*(1 .- a)
    c = map(t->(X[t,:]*X[t,:]'),1:N)
    temp = sum(b.*c)
    for i=1:D
        H[i,:] = temp[i,:]
    end
end

h! (generic function with 1 method)

In [56]:
θ = zeros(D)

3-element Array{Float64,1}:
 0.0
 0.0
 0.0

In [57]:
cost(θ)

0.6931471805599453

In [58]:
result=optimize(cost,g!,h!,θ,LBFGS())

 * Status: success

 * Candidate solution
    Final objective value:     2.034977e-01

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 2.35e-06 ≰ 0.0e+00
    |x - x'|/|x'|          = 9.36e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 5.08e-15 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.50e-14 ≰ 0.0e+00
    |g(x)|                 = 8.58e-10 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    16
    f(x) calls:    58
    ∇f(x) calls:   58


In [59]:
θ_best = Optim.minimizer(result)

3-element Array{Float64,1}:
 -25.161333563346147
   0.2062317132637092
   0.20147160041613388

In [40]:
cost(θ_best)

0.20349770158943997

In [41]:
prediction = map(i->classify(X[i,:], θ_best), 1:N)
accuracy = sum(y .== prediction)/N

0.89

In [42]:
result=optimize(cost,g!,θ,NewtonTrustRegion())

 * Status: success

 * Candidate solution
    Final objective value:     2.034977e-01

 * Found with
    Algorithm:     Newton's Method (Trust Region)

 * Convergence measures
    |x - x'|               = 2.70e-03 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.07e-04 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.09e-09 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.37e-09 ≰ 0.0e+00
    |g(x)|                 = 7.10e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    9
    f(x) calls:    10
    ∇f(x) calls:   10
    ∇²f(x) calls:  10
